<i>STATUS: Draft<i>

In [8]:
import numpy as np
import sympy as sp
from IPython.display import HTML
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.patches import Rectangle
from IPython.display import Image
import sys
import HTM_Code as hc

In the last notebook, we built our first encoder. Admittedly, it was very simple. It just allows us to create integers and provide them to, and provide back indice location. It provided a little extra functionality so it could track previous, and comparison. And we established some rules that we would like to see in place. 

There are range of different encoders that we can use. Obviously more than just numbers. Let's build another simpler one now, called a Random Distributed Scalar Encoder

At this point the amount of code in the notebook is getting a little busy, so we will start bringing it in. In the HTM_Code.py file in the same directory, I have all the functions we have created so far, and we have them here. Going forward, as we develop new functions and classes, we will keep them in this file (later we will need to think about distributing the contents in a more logical way).

So far in our code we have 4 types of things: 

1. SDR Functions (creating SDRs, randomly flipping bits etc.)
2. Metrics functions (compute union and overlap, olap set cardinality, comparing multiple over iterations)
3. Visualisation functions (changing our arrays into matrices for visualisation
4. Classes (1 so far, an encoder class)

When I have created this, I have also cleaned up some functinos. I wanted teh visualisation to generalise to any shape

We also need to start bing more careful of nomenclature which is subtle differently. I am gonig to stry and stick to BAMI, noting htm.core different 

https://numenta.com/assets/pdf/biological-and-machine-intelligence/BaMI-Encoders.pdf

Its important to realise that the earlier things are sill much in play - so we need to consider noise, fault tolerance, and we will do this in the context the RDSE


<div style="background:#99ddff; color:black; padding: 10px">
<b>Add to these notes:</b>

I want to keep these notes in HTML so I don't have to host on a server, but a great exceriise is to use the ipython widgets to experience this like it happens in the video. 
</div>

Let's build another encoder now, and we can explain as we use it: 

In [4]:
class RandomDistributedScalarEncoder:
    def __init__(self, bit_space_size = None,
                number_of_bits_per_value = None,
                min_val = None,
                max_val = None):
        self.bit_space_size = bit_space_size
        self.number_of_bits_per_value = number_of_bits_per_value
        self.unique_bits_per_encoded_value = 1
        self.min_value_to_encode = min_val
        self.max_value_to_encode = max_val
        self.buckets = self.compute_bucket_capacity(self.bit_space_size, self.number_of_bits_per_value)
        self.max_bucket_starting_indice = self.buckets - 1
        self.offset = 1
        self.current_encoded_value = None
        self.bit_locations_for_current_encoded_value = 1
        self.previously_encoded_values = np.array([])
        self.bit_locations_for_previously_encoded_values = []
        self.current_and_previous_encoded_values_comparison = None
        self.is_periodic = False
        
    def get_summary(self):
        print("----------------- SUMMARY -------------------------")
        print("Bit Space Size: ", self.bit_space_size)
        print("Number of bits used to encode each value:", self.number_of_bits_per_value)
        print("Range of values to be encoded: From ", self.min_value_to_encode, ' to ', self.max_value_to_encode)
        print("Number of unique bits per encoded value: ", self.unique_bits_per_encoded_value)
        print("Number of buckets available in bit space:", self.buckets)
        print("Max bucket starting indice:", self.max_bucket_starting_indice)
        print("Current encoded value: ", self.current_encoded_value)
        print("Previously encoded values: ", self.previously_encoded_values)
        print("Bit locations for current encoded value: ", self.bit_locations_for_current_encoded_value)
        print("Bit locations for previously encoded values: ", self.bit_locations_for_previously_encoded_values)
        print("Current and previous encoded values comparison: ", self.current_and_previous_encoded_values_comparison)
        print("Encode periodically: ", self.is_periodic)
    
#     def compute_bucket_capacity(self, n, w):
#         return(n - w + 1)

#     def compare_current_and_previously_encoded_value(self):
#         self.current_and_previous_encoded_values_comparison = np.abs(self.current_encoded_value / self.bit_space_size - self.previously_encoded_values[0] / self.bit_space_size)
    
#     def encode_value_in_bit_space(self, value_choice):
        
#         # check if value is outside range
#         if (value_choice < self.min_value_to_encode) or (value_choice > self.max_value_to_encode):
#             print('Not a valid value to encode as is outside specified range')
#             return()
        
#         if self.current_encoded_value is not None:
#             self.previously_encoded_values = np.append(self.previously_encoded_values, self.current_encoded_value)
#             self.bit_locations_for_previously_encoded_values.append(self.bit_locations_for_current_encoded_value)
#             self.compare_current_and_previously_encoded_value()
#         self.current_encoded_value = value_choice
        
#         if (value_choice > self.max_bucket_starting_indice):
#             value_to_encode = self.max_bucket_starting_indice
#         else:
#             value_to_encode = value_choice
            
#         window = [value_to_encode, value_to_encode + self.number_of_bits_per_value - self.offset]
#         all_values = np.arange(window[0], window[1] + self.offset)
#         self.bit_locations_for_current_encoded_value = all_values
    


In [7]:
b_size = 64
population = 5
min_value_to_encode = 0
max_value_to_encode = 100

# create encoder
encoder1 = RandomDistributedScalarEncoder(bit_space_size = b_size,
                number_of_bits_per_value = population,
                min_val = min_value_to_encode,
                max_val = max_value_to_encode)
encoder1.get_summary()

----------------- SUMMARY -------------------------
Bit Space Size:  64
Number of bits used to encode each value: 5
Range of values to be encoded: From  0  to  100
Number of unique bits per encoded value:  1
Number of buckets available in bit space: 60
Max bucket starting indice: 59
Current encoded value:  None
Previously encoded values:  []
Bit locations for current encoded value:  1
Bit locations for previously encoded values:  []
Current and previous encoded values comparison:  None
Encode periodically:  False


In [9]:
def compute_union_and_overlap(SDR1_on_bits, SDR2_on_bits):
    union = list(set(SDR1_on_bits).union(SDR2_on_bits))
    overlap = list(set(SDR1_on_bits).intersection(SDR2_on_bits))
    
    return({"union": union, "overlap": overlap})



Let's look at more encoders: 
https://numenta.com/assets/pdf/biological-and-machine-intelligence/BaMI-Encoders.pdf

IMportant to capture semantics properly

4 principles recall..... eg consider two numbers should be semantically similiar 

we need encoders to have to to incorporate noise and subsamplinig 

Encoding Daa for HTM systems - Purdy


Date Time Encoder

 
